In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'landscape-pictures:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F298806%2F1217826%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240206%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240206T115724Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D8366a8afe309c500abc31625f4098cb9cf4aa481e19518083781d1718db841888411066447950ca7bad0c5ccbb5fb7a679479b68bc1337b771ba3b11adb364ba887f3b12aa17026e75d633c20f672de96cce9c4984c647db74c8b06fb3587b2e92a9eb379797c732823caae55c546e9a556fd5dfe22b34fbed71fe0cf3c11534d517980ef338c87c77adb92b96cba70b1cd4f46f1ce529a2be8e1f127a21a6b0f766ed63e67731ffca065f51d0749438c2a3f3452a0904c94a89936fecba5ba1efc5463762cb5ba9428fd8288ceba553991e659287e381e0fcf6617cab2b3b1cd1f414b239e4e264d5e42bdebafaf45ad5af95a2cfc3ac5be637c17f95771612'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image, ImageFile
import shutil
from sklearn.model_selection import train_test_split
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
import cv2

In [ ]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from keras.models import Model, load_model,Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Dense, UpSampling2D, RepeatVector, Reshape
from keras.layers import Dropout, Lambda
from keras.layers import Conv2D, Conv2DTranspose
from keras.layers import MaxPooling2D
from keras.layers import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras import backend as K

from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb
from skimage.transform import resize

import tensorflow as tf
from tensorflow.keras.utils import img_to_array, load_img
from tensorflow.keras.utils import plot_model

In [ ]:
# Load full train and test sets
train_imgs = np.load("/kaggle/input/landscapes/train_set.npy")
test_imgs = np.load("/kaggle/input/landscapes/test_set.npy")

print(train_imgs.shape)
print(test_imgs.shape)

In [ ]:
TRAIN_SIZE = 600
TEST_SIZE = 100
EPOCHS = 1200
batch_size = 64

In [ ]:
c_train = (train_imgs[:TRAIN_SIZE])/255
c_test = (test_imgs[:TEST_SIZE])/255

print(c_train.shape)
print(c_test.shape)

In [ ]:
for i, img in enumerate(c_train[:5]):
    plt.subplot(1, 5, i+1)
    plt.imshow(img)
    plt.axis('off')
plt.show()


In [ ]:
inception = InceptionResNetV2(include_top=True)

In [ ]:
def execution():
    embed_input = Input(shape=(1000,))

    encoder_input = Input(shape=(224, 224, 1,))

    encoder_output = Conv2D(64, (3,3), activation='relu', padding='same', strides=2)(encoder_input)
    encoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(encoder_output)
    encoder_output = Conv2D(128, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
    encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)
    encoder_output = Conv2D(256, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
    encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
    encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
    encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)

    fusion_output = RepeatVector(28 * 28)(embed_input)
    fusion_output = Reshape(([28, 28, 1000]))(fusion_output)
    fusion_output = concatenate([encoder_output, fusion_output], axis=3)
    fusion_output = Conv2D(256, (1, 1), activation='relu', padding='same')(fusion_output)

    decoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(fusion_output)

    #decoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(encoder_output)

    decoder_output = UpSampling2D((2, 2))(decoder_output)
    decoder_output = Conv2D(64, (3,3), activation='relu', padding='same')(decoder_output)
    decoder_output = UpSampling2D((2, 2))(decoder_output)
    decoder_output = Conv2D(32, (3,3), activation='relu', padding='same')(decoder_output)
    decoder_output = Conv2D(16, (3,3), activation='relu', padding='same')(decoder_output)
    decoder_output = Conv2D(2, (3, 3), activation='tanh', padding='same')(decoder_output)
    decoder_output = UpSampling2D((2, 2))(decoder_output)

    return Model(inputs=[encoder_input, embed_input], outputs=decoder_output)
    #return Model(inputs=encoder_input, outputs=decoder_output)

In [ ]:
model = execution()
model.compile(optimizer='adam', loss='mse')
model.summary()

In [ ]:
plot_model(model, show_shapes = True)

In [ ]:
def inception_embedding(bw_imgs):
    grayscaled_rgb_resized = []
    for i in bw_imgs:
        i = resize(i, (299, 299))
        grayscaled_rgb_resized.append(i)
    grayscaled_rgb_resized = np.array(grayscaled_rgb_resized)
    grayscaled_rgb_resized = preprocess_input(grayscaled_rgb_resized)
    embed = inception.predict(grayscaled_rgb_resized)
    return embed

In [ ]:
# Image transformer
datagen = ImageDataGenerator(
        shear_range=0.4,
        zoom_range=0.4,
        rotation_range=40,
        horizontal_flip=True)

#Generate training data


In [ ]:
def create_input(size):
    for batch in datagen.flow(c_train, batch_size=batch_size):
        grayscaled_rgb = gray2rgb(rgb2gray(batch))
        embed = inception_embedding(grayscaled_rgb)
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        X_batch = X_batch.reshape(X_batch.shape+(1,))
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield ([X_batch, inception_embedding(grayscaled_rgb)], Y_batch)

In [ ]:
test_input = gray2rgb(rgb2gray(c_test))
test_embed = inception_embedding(test_input)
test_input = rgb2lab(test_input)[:,:,:,0]
test_input = test_input.reshape(test_input.shape+(1,))

In [ ]:
model.fit(create_input(batch_size), steps_per_epoch=1, epochs=EPOCHS)

In [ ]:
output = (model.predict([test_input, test_embed])) * 128

In [ ]:
os.makedirs('/kaggle/working/my_images', exist_ok=True)

In [ ]:
mean_SSIM = 0
mean_PSNR = 0

# Iterate over the test samples
for i in range(len(test_input)):
    # Create a figure with subplots
    print(f"\n-------------- Image {i} --------------------------")

    fig, axes = plt.subplots(1, 3, figsize=(18, 6))

    # Original black and white image
    axes[0].imshow(test_input[i], cmap='gray')
    axes[0].set_title('Original BW Image')
    axes[0].axis('off')

    # True colored image (replace 'true_color_images' with your actual data)
    axes[1].imshow(c_test[i])
    axes[1].set_title('True Color Image')
    axes[1].axis('off')

    # Colorized image from the model
    canvas = np.zeros((224, 224, 3))
    canvas[:,:,0] = test_input[i][:,:,0]
    canvas[:,:,1:] = output[i]
    colorized_image = lab2rgb(canvas)
    axes[2].imshow(colorized_image)
    axes[2].set_title('Colorized Image')
    axes[2].axis('off')

    PSNR = psnr(c_test[i],colorized_image)
    mean_PSNR += PSNR
    print(f"PSNR {i}: {PSNR} db")

    SSIM = ssim(c_test[i], colorized_image, data_range=1, channel_axis=-1)
    mean_SSIM += SSIM
    print(f"SSIM {i}: {SSIM}")


    # Display the figure
    plt.show()

    cv2.imwrite(f'/kaggle/working/my_images/image_{i}.jpg', colorized_image*255)


In [ ]:
print(f"Mean PSNR for the test set: {mean_PSNR/TEST_SIZE} db")
print(f"Mean SSIM for the test set: {mean_SSIM/TEST_SIZE}")